# Strands Agent에서 Model Context Protocol (MCP)을 도구로 사용하기

## 개요
[Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction)은 애플리케이션이 대규모 언어 모델(LLM)에 컨텍스트를 제공하는 방법을 표준화하는 개방형 프로토콜입니다. Strands AI SDK는 MCP와 통합하여 외부 도구와 서비스를 통해 에이전트 기능을 확장합니다.

MCP는 추가 도구를 제공하는 MCP 서버와 에이전트 간의 통신을 가능하게 합니다. Strands Agent SDK에는 MCP 서버에 연결하고 해당 도구를 사용하기 위한 내장 지원이 포함되어 있습니다.

이 예제에서는 Strands Agent에서 MCP 도구를 사용하는 방법을 보여드리겠습니다. AWS 문서에 액세스하고, 콘텐츠를 검색하고, 권장 사항을 얻는 도구를 제공하는 [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)를 사용하겠습니다. 이 MCP 서버에는 3가지 주요 기능이 있습니다:

- **문서 읽기**: AWS 문서 페이지를 가져와서 마크다운 형식으로 변환
- **문서 검색**: 공식 검색 API를 사용하여 AWS 문서 검색
- **권장 사항**: AWS 문서 페이지에 대한 콘텐츠 권장 사항 가져오기



## 에이전트 세부사항
<div style="float: left; margin-right: 20px;">
    
|기능                |설명                                               |
|--------------------|---------------------------------------------------|
|사용된 기능          |MCP 도구                                           |
|에이전트 구조        |단일 에이전트 아키텍처                              |

</div>

## 아키텍처

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## 주요 기능
* **단일 에이전트 아키텍처**: 이 예제는 MCP 도구와 상호작용하는 단일 에이전트를 생성합니다
* **MCP 도구**: 에이전트와 MCP 도구의 통합

## 설정 및 사전 요구사항

### 사전 요구사항
* Python 3.10+
* AWS 계정
* Amazon Bedrock에서 Anthropic Claude 3.7 활성화

이제 Strands Agent에 필요한 패키지들을 설치하겠습니다

In [ ]:
# 사전 요구사항 설치
%pip install -r requirements.txt

### 의존성 패키지 가져오기

이제 의존성 패키지들을 가져오겠습니다

In [ ]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### stdio 전송을 사용하여 MCP 서버에 연결

MCP의 [전송(Transports)](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports)은 클라이언트와 서버 간의 통신 기반을 제공합니다. 메시지가 전송되고 수신되는 방법의 기본 메커니즘을 처리합니다. 현재 MCP에 내장된 세 가지 표준 전송 구현이 있습니다:

- **표준 입력/출력 (stdio)**: 표준 입력 및 출력 스트림을 통한 통신을 가능하게 합니다. 로컬 통합 및 명령줄 도구에 특히 유용합니다
- **스트리밍 가능한 HTTP**: 이전 프로토콜 버전의 HTTP+SSE 전송을 대체합니다. 스트리밍 가능한 HTTP 전송에서 서버는 여러 클라이언트 연결을 처리할 수 있는 독립적인 프로세스로 작동합니다. 이 전송은 HTTP POST 및 GET 요청을 사용합니다. 서버는 선택적으로 Server-Sent Events (SSE)를 사용하여 여러 서버 메시지를 스트리밍할 수 있습니다. 이를 통해 기본 MCP 서버뿐만 아니라 스트리밍 및 서버-클라이언트 알림 및 요청을 지원하는 더 기능이 풍부한 서버를 허용합니다.
- **SSE**: Server-Sent Events 전송을 사용하는 HTTP 기반 MCP 서버용 레거시 전송

전반적으로 명령줄 도구 구축, 로컬 통합 구현 및 셸 스크립트 작업에는 stdio를 사용해야 합니다. 상태 비저장 통신을 처리하거나 리소스 사용량 최소화가 중요할 때 특히 AI 에이전트가 도구 및 서비스와 통신하는 유연하고 효율적인 방법이 필요한 경우 스트리밍 가능한 HTTP 전송을 사용해야 합니다.

특정 요구 사항에 맞는 **사용자 정의 전송** 구현을 사용할 수도 있습니다.


이제 stdio 전송을 사용하여 MCP 서버에 연결해보겠습니다. 먼저 `MCPClient` 클래스를 사용하여 [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)에 연결하겠습니다. 이 서버는 AWS 문서에 액세스하고, 콘텐츠를 검색하고, 권장 사항을 얻는 도구를 제공합니다.

In [ ]:
# stdio 전송을 사용하여 MCP 서버에 연결
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### 에이전트 구성 설정 및 호출

다음으로 방금 생성한 `stdio_mcp_client` 객체의 도구를 사용하여 에이전트 구성을 설정하겠습니다. 이를 위해 MCP 서버에서 사용 가능한 도구를 나열해야 합니다. 이를 위해 `list_tools_sync` 메서드를 사용할 수 있습니다.

그 후 에이전트에게 질문을 하겠습니다.

In [ ]:
# MCP 도구로 에이전트 생성
with stdio_mcp_client:
    # MCP 서버에서 도구 가져오기
    tools = stdio_mcp_client.list_tools_sync()

    # 이러한 도구로 에이전트 생성
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("Amazon Bedrock의 가격 모델은 무엇인가요? 간결하게 답해주세요.")

### 스트리밍 가능한 HTTP를 사용하여 MCP 서버에 연결

이제 스트리밍 가능한 HTTP 전송을 사용하여 MCP 서버에 연결해보겠습니다. 먼저 스트리밍 가능한 HTTP 전송을 사용하여 간단한 MCP 서버를 시작하겠습니다.

이 예제에서는 자체 MCP 서버를 생성하겠습니다. 아키텍처는 다음과 같습니다

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [ ]:
# MCP 서버 생성
mcp = FastMCP("Calculator Server")

# 도구 정의


@mcp.tool(description="계산을 수행하는 계산기 도구")
def calculator(x: int, y: int) -> int:
    """두 숫자를 더합니다."""
    return x + y


@mcp.tool(description="오래 실행되는 도구입니다")
def long_running_tool(name: str) -> str:
    """오래 실행되는 작업을 시뮬레이션합니다."""
    time.sleep(25)
    return f"안녕하세요 {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

이제 `streamable-http` 서버로 스레드를 시작하겠습니다

In [ ]:
thread = threading.Thread(target=main)
thread.start()

#### 스트리밍 가능한 HTTP 클라이언트와 에이전트 통합

이제 `streamablehttp_client`를 사용하여 이 서버를 간단한 에이전트와 통합해보겠습니다.

In [ ]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### 에이전트 구성 설정 및 호출

다음으로 방금 생성한 `streamable_http_mcp_client` 객체의 도구를 사용하여 에이전트 구성을 설정하겠습니다. 이를 위해 MCP 서버에서 사용 가능한 도구를 나열해야 합니다. 이를 위해 `list_tools_sync` 메서드를 사용할 수 있습니다.

그 후 에이전트에게 질문을 하겠습니다.

In [ ]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("2 + 2는 무엇인가요?"))

### 직접 도구 호출

도구는 일반적으로 사용자 요청에 따라 에이전트에 의해 호출되지만, MCP 도구를 직접 호출할 수도 있습니다. 이는 여러 도구를 함께 조율하는 워크플로 시나리오에 유용할 수 있습니다.

In [ ]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # 직접 도구 호출
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # 결과 처리
    print(f"계산 결과: {result['content'][0]['text']}")

MCP 서버 도구를 호출할 때 너무 오래 실행되지 않도록 `read_timeout_seconds`를 선택적으로 제공할 수도 있습니다

In [ ]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"도구 실행 실패: {result['content'][0]['text']}")
        else:
            print(f"도구 실행 성공: {result['content'][0]['text']}")
    except Exception as e:
        print(f"도구 호출 시간 초과 또는 실패: {str(e)}")

### 여러 MCP 서버와 상호작용

Strands Agents를 사용하면 동일한 에이전트를 사용하여 여러 MCP 서버와 상호작용하고 병렬로 사용할 수 있는 최대 도구 수(`max_parallel_tools`)와 같은 도구 설정을 구성할 수도 있습니다. 이 구성을 보여주기 위해 새 에이전트를 만들어보겠습니다:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

이 에이전트에서는 AWS Documentation MCP 서버를 다시 사용하고 AWS Cloud Development Kit (CDK) 모범 사례, 코드형 인프라 패턴 및 CDK Nag를 통한 보안 규정 준수를 지원하는 [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/)도 사용하겠습니다.

먼저 stdio 전송을 사용하여 두 MCP 서버에 연결해보겠습니다

In [ ]:
# stdio 전송을 사용하여 MCP 서버에 연결
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# stdio 전송을 사용하여 MCP 서버에 연결
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### MCP 서버로 에이전트 생성

다음으로 두 MCP 서버의 도구로 에이전트를 생성하겠습니다

In [ ]:
# MCP 도구로 에이전트 생성
with aws_docs_mcp_client, cdk_mcp_client:
    # MCP 서버에서 도구 가져오기
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # 이러한 도구로 에이전트 생성
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "Amazon Bedrock의 가격 모델은 무엇인가요? 간결하게 답해주세요. 또한 CDK와 관련된 모범 사례는 무엇인가요?"
    )

### 축하합니다!

이 노트북에서 Strands Agent를 사용하여 MCP 서버에 연결하고 두 가지 MCP 전송 프로토콜인 stdio와 스트리밍 가능한 HTTP를 사용하는 방법을 배웠습니다. 또한 동일한 에이전트에 여러 MCP 서버를 연결하는 방법도 배웠습니다. 다음으로 에이전트와 함께 다양한 모델을 사용하는 방법을 살펴보겠습니다